# Tool Calling: Extending What Agents Can Do

In the previous chapter, you built AI agents that used tools like Calculator and Wikipedia. The agent decided *when* to call each tool based on the user's question. This chapter goes deeper: how do tools actually work, how do you design them well, and how do you keep agents safe?

**What you'll learn:**
- How tool calling works under the hood
- The critical difference between read-only and write tools
- How to write tool descriptions that actually work
- Guardrails to prevent agents from causing problems
- How to build a complete research-to-action workflow with approval

---

## Workflows vs Agents: A Quick Review

Before diving into tools, let's clarify when to use each approach:

| Approach | Who decides the steps? | Best for |
|----------|------------------------|----------|
| **Workflow** | You (the builder) | Predictable tasks with known steps |
| **Agent** | The model | Exploratory tasks where steps vary |

**The rule:** If you can write out all the IF statements → use a workflow. If the path depends on unpredictable user input → consider an agent.

**Example:**
- "Classify this ticket and route to the right team" → **Workflow** (you know the categories)
- "Research this topic and answer follow-up questions" → **Agent** (questions are unpredictable)

This chapter focuses on making agents work *safely* by understanding their tools.

---

## How Tool Calling Works

When you connect tools to an AI Agent node, you're giving the model a menu of capabilities. The model doesn't execute tools directly—it *requests* that a tool be called, and n8n executes it.

### The Loop

```
┌─────────────────────────────────────────────────────────────┐
│                                                             │
│   ┌─────────┐     ┌─────────┐     ┌─────────┐              │
│   │ REASON  │────▶│  CALL   │────▶│ OBSERVE │──┐           │
│   │         │     │  TOOL   │     │ RESULT  │  │           │
│   └─────────┘     └─────────┘     └─────────┘  │           │
│        ▲                                       │           │
│        │                                       │           │
│        └───────────────────────────────────────┘           │
│                    (repeat if needed)                      │
│                                                             │
│   When done: ───▶ FINAL ANSWER                             │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

1. **Reason** — The model reads the user's question and its available tools
2. **Call Tool** — The model outputs which tool to call and with what inputs
3. **Observe** — n8n executes the tool and returns the result to the model
4. **Repeat or Answer** — The model decides if it needs more tools or can answer

This is called the **ReAct pattern** (Reasoning + Acting).

### Why It Matters

The model chooses tools based on:
- The **tool name** and **description** you provide
- The **user's question**
- The **system message** instructions

If any of these are unclear, the model may call the wrong tool, call no tool, or hallucinate an answer instead of looking it up.

---

## Anatomy of a Tool

Every tool has five key parts:

| Part | What it is | Example (Wikipedia) |
|------|------------|---------------------|
| **Name** | Short identifier | `wikipedia` |
| **Description** | When to use it | "Search Wikipedia for factual information" |
| **Inputs** | What it needs | `query` (string) |
| **Outputs** | What it returns | Article text or summary |
| **Side Effects** | Does it change anything? | No (read-only) |

The **description** is the most important part. The model reads it to decide whether to use the tool. A vague description leads to wrong decisions.

### Good vs Bad Descriptions

| Bad | Good |
|-----|------|
| "A calculator" | "Use for ANY arithmetic. Input: math expression. Output: numeric result." |
| "Searches things" | "Search Wikipedia for factual information about people, places, events, or dates. Use when the user asks about real-world facts." |
| "Sends email" | "Draft and send an email. ONLY use when user explicitly requests sending a message. Requires: recipient, subject, body." |

**Tip from OpenAI:** "Write tool descriptions as if explaining to an intern who knows nothing about your system."

---

## Read-Only vs Write Tools

This is the most important distinction in tool design:

| Type | What it does | Risk | Default policy |
|------|--------------|------|----------------|
| **Read-only** | Fetches or calculates data | Low | Allow freely |
| **Write** | Changes something in the real world | High | Require approval |

### Read-Only Tools (Safe)

- Wikipedia lookup
- Calculator
- Weather API
- Database query (SELECT only)
- Web search

These tools only *read* information. If the agent calls them incorrectly, nothing bad happens—you just get wrong or irrelevant data.

### Write Tools (Dangerous)

- Send email
- Create ticket
- Update database
- Post to social media
- Delete file

These tools *change* something. If the agent calls them incorrectly, you could send embarrassing emails, create duplicate tickets, or lose data.

### The Golden Rule

> **If a tool changes something in the real world, it needs human approval before executing.**

This is why the Human-in-the-Loop pattern from Chapter 5 is so important. We'll use it again in this chapter's workflow.

---

## Writing Tool Descriptions That Work

Tool descriptions are where you have the most control over agent behavior. Here's a template:

```
Use this tool when [specific situation].
Do NOT use when [counter-situation].
If [required info] is missing, ask the user first.
Input: [parameter names and types]
Output: [what it returns]
```

### Example: Wikipedia Tool

**Bad description:**
```
Searches Wikipedia.
```

**Good description:**
```
Use this tool when the user asks about real-world facts: people, places, 
historical events, dates, or definitions.

Do NOT use for opinions, predictions, or recent news (Wikipedia may be outdated).

Input: search query (string)
Output: article summary text
```

### Example: Send Email Tool

**Bad description:**
```
Sends an email.
```

**Good description:**
```
Use this tool ONLY when the user explicitly asks to send an email or message.

Do NOT use to "draft" or "prepare" — use Draft Email for that.
Do NOT use if recipient, subject, or body is missing — ask the user first.

Input: recipient (email), subject (string), body (string)
Output: confirmation message

WARNING: This action cannot be undone. Email will be sent immediately.
```

### Testing Your Descriptions

Try these test prompts with different descriptions and observe which tool the agent chooses:

| Prompt | Expected tool |
|--------|---------------|
| "Who invented the telephone?" | Wikipedia |
| "What's 15% of 340?" | Calculator |
| "Send a thank-you email to John" | Send Email |
| "Help me write an email" | Draft Email (not Send) |

If the agent picks the wrong tool, improve the description.

```{note}
**What if you have many tools?**

This course uses 2-3 tools per agent. In production systems with 10+ tools, new challenges appear: the agent may pick the wrong tool, calls become slow and expensive, or tools from different domains interfere with each other.

Common solutions:
- **Tool Search**: Retrieve only relevant tools per request (instead of loading all 50)
- **Programmatic Tool Calling**: Agent writes code that calls multiple tools in batch, returns only the result
- **Universal Code Runner**: One powerful Code node instead of many small tools (n8n's Code node is exactly this)
- **Router → Specialists**: Split into multiple focused agents (e.g., Support Agent, Billing Agent) with isolated tool sets
- **Tool Use Examples**: Add example calls inside tool descriptions to improve parameter accuracy

For implementation details, see [Anthropic's Advanced Tool Use](https://www.anthropic.com/engineering/advanced-tool-use).
```

---

## Guardrails: Preventing Problems

Even with good descriptions, agents can misbehave. Here are the guardrails you should implement:

### 1. Max Iterations

The AI Agent node has a **Max Iterations** setting (default: 10). This prevents infinite loops where the agent keeps calling tools without making progress.

**Signs of a looping agent:**
- Same tool called repeatedly with similar inputs
- Agent says "let me try again" multiple times
- Execution takes unusually long

**Fix:** Lower max iterations and improve your system message.

### 2. System Message Boundaries

Tell the agent explicitly what it should NOT do:

```
Rules:
- NEVER send an email without user confirmation.
- If you don't know something, say so — don't guess.
- Call Wikipedia at most twice per question.
- If a tool fails, explain the error and stop.
```

### 3. Approval Gates for Write Actions

Use a **Wait node** before any write action:

```
Agent drafts email → Store draft → WAIT for approval → Send email
```

This pattern ensures a human reviews the output before it affects the real world.

### 4. Separate Read and Write Permissions

If possible, give agents only the tools they need:
- A "research assistant" agent only needs read tools
- A "customer support" agent might need write tools, but with approval gates

**Principle of least privilege:** Start with read-only, add write only when necessary.

---

## Putting It Together: Research Agent with Approval

This workflow demonstrates everything we've learned:
- An agent with **read-only tools** (Wikipedia, Calculator)
- An **approval gate** before any write action could happen
- Good **tool descriptions** and **guardrails**

We use familiar tools (Wikipedia, Calculator) so you can focus on the new concepts: safety patterns, tool contracts, and approval gates.

```
┌──────────────────┐     ┌──────────────────┐     ┌──────────────────┐
│  Manual Trigger  │────▶│  Input: Topic    │────▶│   AI Agent       │
└──────────────────┘     └──────────────────┘     │  (Research)      │
                                                  └──────────────────┘
                                                          │
                                          ┌───────────────┴───────────────┐
                                          │                               │
                                    [Wikipedia]                    [Calculator]
                                    (read-only)                    (read-only)
                                          │
                                          ▼
                                ┌──────────────────┐
                                │  Format Summary  │
                                │  + Draft Email   │
                                └──────────────────┘
                                          │
                                          ▼
                                ┌──────────────────┐
                                │  ⏸️ WAIT for    │
                                │    Approval     │
                                └──────────────────┘
                                          │
                                    Human approves
                                          │
                                          ▼
                                ┌──────────────────┐
                                │  Output: Ready   │
                                │  to Send         │
                                └──────────────────┘
```

### The Mission

1. User provides a research topic
2. Agent looks up information using Wikipedia
3. Agent calculates any numbers if needed
4. Workflow formats findings as an email draft
5. **Human reviews and approves** the draft

**Note:** This workflow stops at approval. In production, you would connect a Gmail/Outlook node after the Wait to actually send the email. The pattern is: AI drafts → human approves → system sends.

---

### Import the Workflow

> **Import via URL** (copy and paste in n8n → Import from URL):
> ```
> https://raw.githubusercontent.com/ezponda/ai-agents-course/main/courses/n8n_no_code/book/_static/workflows/09_tool_calling_research_approval.json
> ```
>
> **Download:** {download}`09_tool_calling_research_approval.json <_static/workflows/09_tool_calling_research_approval.json>`

::::{dropdown} 🛠️ Build this workflow from scratch (step-by-step)
:color: secondary

### Step 1: Create the workflow

1. Click **Workflows** → **Add Workflow**
2. Rename to "Research Agent with Approval"

### Step 2: Add trigger and input

1. Add **Manual Trigger**
2. Add **Edit Fields** → rename to `Input — Research Topic`
3. Add two fields:

| Name | Value |
|------|-------|
| `topic` | `When was the Eiffel Tower built and how tall is it in feet?` |
| `recipient` | `team@example.com` |

### Step 3: Add the AI Agent

1. Add **AI Agent** → rename to `Research Agent`
2. Set **Prompt** (Expression mode):
   ```
   Research this topic and provide a brief summary (3-4 sentences):
   
   {{ $json.topic }}
   
   Use Wikipedia for facts. Use Calculator for any unit conversions.
   ```
3. Set **System Message**:
   ```
   You are a research assistant. Your job is to find accurate information.
   
   Rules:
   - Use Wikipedia to look up facts. Do NOT guess.
   - Use Calculator for any math or conversions.
   - Call each tool at most twice. If you still can't find the answer, explain what you tried.
   - Keep your summary brief: 3-4 sentences maximum.
   - If you can't find information, say so clearly.
   ```

### Step 4: Connect sub-nodes to the Agent

1. Add **OpenRouter Chat Model** (or your preferred provider) → connect to Agent
2. Add **Wikipedia** tool → connect to Agent
3. Add **Calculator** tool → connect to Agent

### Step 5: Format the email draft

1. Add **Edit Fields** → rename to `Format Email Draft`
2. Add fields:

| Name | Value (Expression) |
|------|--------------------|
| `to` | `{{ $node['Input — Research Topic'].json.recipient }}` |
| `subject` | `Research Summary: {{ $node['Input — Research Topic'].json.topic.substring(0, 50) }}...` |
| `body` | `{{ $json.output }}` |
| `drafted_at` | `{{ $now.format('yyyy-MM-dd HH:mm') }}` |
| `status` | `pending_approval` |

### Step 6: Add the approval gate

1. Add **Wait** → rename to `Wait — Approval`
2. Set **Resume**: `On Webhook Call`
3. In Options, set **Webhook Suffix**: `{{ $execution.id }}`

### Step 7: Add the output

1. Add **Edit Fields** → rename to `Output — Approved`
2. Add fields:

| Name | Value (Expression) |
|------|--------------------|
| `email_to` | `{{ $node['Format Email Draft'].json.to }}` |
| `email_subject` | `{{ $node['Format Email Draft'].json.subject }}` |
| `email_body` | `{{ $node['Format Email Draft'].json.body }}` |
| `status` | `approved` |
| `approved_at` | `{{ $now.format('yyyy-MM-dd HH:mm') }}` |

3. Enable **Keep Only Set**

### Step 8: Test

1. Click **Execute Workflow**
2. Watch the agent use Wikipedia and Calculator
3. See the draft email at "Format Email Draft"
4. Copy the **Test URL** from the Wait node
5. Open the URL in your browser → workflow completes

::::

### Node-by-Node Walkthrough

| Node | Type | What it does |
|------|------|-------------|
| **Run: Research** | Manual Trigger | Starts the workflow |
| **Input — Research Topic** | Set | Provides topic and recipient |
| **Research Agent** | AI Agent | Looks up facts using Wikipedia and Calculator |
| **Format Email Draft** | Set | Structures the agent's output as an email |
| **Wait — Approval** | Wait | Pauses until human approves |
| **Output — Approved** | Set | Final output ready for sending |

**Sub-nodes connected to Research Agent:**
- OpenRouter Chat Model (language model)
- Wikipedia (read-only tool)
- Calculator (read-only tool)

### System Message

```
You are a research assistant. Your job is to find accurate information.

Rules:
- Use Wikipedia to look up facts. Do NOT guess.
- Use Calculator for any math or conversions.
- Call each tool at most twice. If you still can't find the answer, explain what you tried.
- Keep your summary brief: 3-4 sentences maximum.
- If you can't find information, say so clearly.
```

**Note:** The "at most twice" rule is a practical guardrail that prevents the agent from looping endlessly on difficult queries.

### Test Prompts to Try

| Topic | What happens |
|-------|-------------|
| `When was the Eiffel Tower built and how tall is it in feet?` | Wikipedia lookup + Calculator conversion |
| `Who founded Microsoft and what year? Calculate how many years ago that was.` | Wikipedia + Calculator |
| `What is the population of Japan?` | Wikipedia only |
| `What is 15% of 2,340?` | Calculator only |

### Data Flow

```
INPUT                          OUTPUT
─────                          ──────
Trigger: { }
    ↓
Input: { topic: "When was Eiffel Tower built...", recipient: "team@..." }
    ↓
Agent: [calls Wikipedia] → [calls Calculator] → { output: "The Eiffel Tower was built in 1889..." }
    ↓
Format Email Draft: {
  to: "team@example.com",
  subject: "Research Summary: When was the Eiffel Tower built...",
  body: "The Eiffel Tower was built in 1889...",
  drafted_at: "2025-01-25 15:30",
  status: "pending_approval"
}
    ↓
⏸️ WAIT — Human reviews the draft
    ↓
(human opens Test URL)
    ↓
Output: {
  email_to: "team@example.com",
  email_subject: "Research Summary: ...",
  email_body: "The Eiffel Tower was built in 1889...",
  status: "approved",
  approved_at: "2025-01-25 15:32"
}
```

**Key insight:** The agent does all the research (read-only), but the email action (write) is blocked until a human approves.

---

## Debugging Tool Calls

When tools don't work as expected, check these in order:

| Problem | What to check | Fix |
|---------|---------------|-----|
| Agent doesn't use any tool | Is the description clear? | Add "Use this tool when..." |
| Agent uses wrong tool | Are descriptions overlapping? | Add "Do NOT use when..." |
| Agent calls tool repeatedly | Is it getting useful results? | Check tool output, improve query |
| Agent ignores tool output | Is system message too restrictive? | Adjust instructions |
| Tool returns error | Are credentials set up? | Check Settings → Credentials |

### Debug Checklist

1. **Click the Agent node** → look at the "Steps" in the output panel
2. **Check which tools were called** and with what inputs
3. **Read the tool outputs** — did they return useful data?
4. **Look at the final answer** — did the agent use the tool output?

---

## MCP: Connecting to External Tool Servers (Optional)

**MCP** (Model Context Protocol) is a standard for connecting AI agents to external tools. Instead of building tools inside n8n, you can connect to tools hosted elsewhere.

n8n has native MCP support with three nodes:

| Node | What it does |
|------|-------------|
| **MCP Client Tool** | Connect your AI Agent to tools from an external MCP server |
| **MCP Client** | Use MCP tools in regular workflows (not agents) |
| **MCP Server Trigger** | Expose your n8n workflows as tools for other MCP clients |

### When to Use MCP

- You want to use tools from services like Notion, Slack, or custom APIs that provide MCP endpoints
- You want to expose your n8n workflows as tools for Claude Desktop, Cursor, or other MCP clients
- You're building a system where multiple AI applications share the same tools

### Example: Connecting to an MCP Server

1. Add **MCP Client Tool** as a sub-node to your AI Agent
2. Configure the **SSE Endpoint** URL of the MCP server
3. Set authentication if required
4. Choose which tools to expose to the agent (All or Selected)

The agent can now use tools from that external server.

```{note}
MCP is powerful but adds complexity. For this course, the built-in tools (Wikipedia, Calculator, HTTP Request) are sufficient. Explore MCP when you need tools that aren't available natively in n8n.

**Learn more:** [n8n MCP Documentation](https://docs.n8n.io/integrations/builtin/cluster-nodes/sub-nodes/n8n-nodes-langchain.toolmcp/)
```

---

## Summary

| Concept | Key Point |
|---------|----------|
| **Tool calling** | Model requests tools; n8n executes them |
| **ReAct loop** | Reason → Call Tool → Observe → Repeat or Answer |
| **Read-only tools** | Safe to use freely (Wikipedia, Calculator) |
| **Write tools** | Need human approval (send email, create ticket) |
| **Descriptions** | "Use when... / Do NOT use when..." format |
| **Guardrails** | Max iterations, system message rules, approval gates |
| **MCP** | Standard for connecting to external tool servers |

---

## Next Steps

You now understand how to give agents tools safely. Key takeaways:

1. **Read-only tools are safe** — let agents use them freely
2. **Write tools are dangerous** — always add approval gates
3. **Descriptions matter** — spend time writing clear "when to use" guidance
4. **Test with edge cases** — see what happens when the agent makes mistakes

The **Node Toolbox** appendix provides reference documentation for all nodes covered in this course. Use it when you need to look up specific parameters or troubleshoot issues.